In [1]:
import pandas as pd
import os

# Load the dataset
file_path = 'D:\\1 code AI'
file_names = ['new_data_v1.csv']
datasets = [pd.read_csv(os.path.join(file_path, file)) for file in file_names]
dataset = pd.concat(datasets, ignore_index=True)

# Count the number of labels 0 and 1
label_counts = dataset['Label'].value_counts()
print("Label counts:")
print(label_counts)

dataset.info()


Label counts:
Label
0    204368
1     34767
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239135 entries, 0 to 239134
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   row     239135 non-null  int64  
 1   col     239135 non-null  int64  
 2   year    239135 non-null  int64  
 3   month   239135 non-null  int64  
 4   day     239135 non-null  int64  
 5   hour    239135 non-null  int64  
 6   B04B    239135 non-null  float64
 7   B05B    239135 non-null  float64
 8   B06B    239135 non-null  float64
 9   B09B    239135 non-null  float64
 10  B10B    239135 non-null  float64
 11  B11B    239135 non-null  float64
 12  B12B    239135 non-null  float64
 13  B14B    239135 non-null  float64
 14  B16B    239135 non-null  float64
 15  I2B     239135 non-null  float64
 16  I4B     239135 non-null  float64
 17  IRB     239135 non-null  float64
 18  VSB     239135 non-null  float64
 19  WVB     239135 n

In [2]:
from sklearn.model_selection import train_test_split

# Check if columns exist before dropping
columns_drop = ["Label", "row", "col", "year", "month", "day", "hour"]
existing_columns_to_drop = [col for col in columns_drop if col in dataset.columns]

X = dataset.drop(columns=existing_columns_to_drop)
y = dataset["Label"]

print(X.head())
print(X.info())
print(y.head())
print(y.value_counts())

       B04B      B05B      B06B       B09B       B10B       B11B       B12B  \
0  0.084788  0.053601  0.030115  253.87752  261.63812  281.48710  260.33334   
1  0.129801  0.103687  0.064551  253.48486  261.10180  279.65836  259.01260   
2  0.088053  0.074731  0.043408  253.62572  261.10180  281.48782  260.33664   
3  0.085429  0.070825  0.040283  253.06673  261.10593  280.73180  259.73105   
4  0.099121  0.091174  0.057520  252.52501  260.70062  280.29822  259.61038   

        B14B       B16B        I2B        I4B        IRB       VSB        WVB  
0  285.00070  269.99298  281.76694  286.36456  285.24005  0.070424  244.27534  
1  282.17892  268.44498  279.95640  286.18780  282.48993  0.099792  244.05240  
2  284.34457  269.33520  281.68080  286.60724  284.50903  0.070424  243.55762  
3  283.88354  269.33356  280.94617  285.77060  284.15130  0.074331  243.33647  
4  283.85870  268.99704  280.34543  287.78340  284.39117  0.082174  242.84953  
<class 'pandas.core.frame.DataFrame'>
RangeIn

In [3]:
# Split the data into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

print("Training set label distribution:")
print(y_train.value_counts())
print("Test set label distribution:")
print(y_test.value_counts())


Training set size: 167394
Test set size: 71741
Training set label distribution:
Label
0    143026
1     24368
Name: count, dtype: int64
Test set label distribution:
Label
0    61342
1    10399
Name: count, dtype: int64


chạy bừa

In [ ]:
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


# Define the model
BRFmodel =  (n_estimators=536, max_depth=50, min_samples_split=4, min_samples_leaf=4, max_features='log2', class_weight='balanced', random_state=42)

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

# Train the model and evaluate using cross-validation
scores = cross_validate(BRFmodel, X_train, y_train, scoring=['accuracy', 'precision', 'recall', 'f1'], cv=cv, n_jobs=-1)

# Print the scores
print("Accuracy: ", scores['test_accuracy'].mean())
print("Precision: ", scores['test_precision'].mean())
print("Recall: ", scores['test_recall'].mean())
print("F1 Score: ", scores['test_f1'].mean())


# Fit the model on the training data
BRFmodel.fit(X_train, y_train)

# Predict on the test data
y_pred = BRFmodel.predict(X_test)

# Generate and print the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['No Rain', 'Rain'], yticklabels=['No Rain', 'Rain'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

tunning

In [15]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': range(100, 501, 100),  # You can adjust the range if needed
    'max_depth': range(10, 51, 10),
    'min_samples_split': range(2, 20, 2),
    'min_samples_leaf': range(2, 20, 2),
    'max_features': [None, 'sqrt', 'log2'],
    'class_weight': ['balanced', 'balanced_subsample']
}

# Initialize the model
BRFmodel = BalancedRandomForestClassifier(random_state=42)

# Define the cross-validation strategy
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

# Initialize the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=BRFmodel, 
    param_distributions=param_grid, 
    n_iter=10, 
    scoring='f1', 
    cv=cv, 
    random_state=42, 
    n_jobs=-1,
    verbose=2
)

# Fit the random search model
random_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", random_search.best_params_)
print("Best F1 Score:", random_search.best_score_)

# Predict on the test data using the best model
y_pred = random_search.best_estimator_.predict(X_test)

# Generate and print the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)


Fitting 15 folds for each of 10 candidates, totalling 150 fits


d:\Enviroment\Anaconda\envs\AI\lib\site-packages\imblearn\ensemble\_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
d:\Enviroment\Anaconda\envs\AI\lib\site-packages\imblearn\ensemble\_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
d:\Enviroment\Anaconda\envs\AI\lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  wa

Best Parameters: {'n_estimators': 500, 'min_samples_split': 14, 'min_samples_leaf': 16, 'max_features': None, 'max_depth': 50, 'class_weight': 'balanced_subsample'}
Best F1 Score: 0.607914312243648
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.83      0.90     40859
           1       0.47      0.85      0.60      6968

    accuracy                           0.84     47827
   macro avg       0.72      0.84      0.75     47827
weighted avg       0.90      0.84      0.85     47827



In [5]:

from imblearn.ensemble import BalancedRandomForestClassifier

# Initialize the classifier with the best parameters
clf = BalancedRandomForestClassifier(
    n_estimators=650,
    min_samples_split=14,
    min_samples_leaf=16,
    max_features=None,
    max_depth=50,
    class_weight='balanced_subsample'
)

# Fit the classifier on the training data
clf.fit(X_train, y_train)

# Predict on the test data
y_pred = clf.predict(X_test)

# Generate the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

d:\Enviroment\Anaconda\envs\AI\lib\site-packages\imblearn\ensemble\_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
d:\Enviroment\Anaconda\envs\AI\lib\site-packages\imblearn\ensemble\_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
d:\Enviroment\Anaconda\envs\AI\lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  wa

NameError: name 'classification_report' is not defined

In [6]:
from sklearn.metrics import classification_report
# Generate the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.84      0.90     61342
           1       0.47      0.85      0.60     10399

    accuracy                           0.84     71741
   macro avg       0.72      0.84      0.75     71741
weighted avg       0.90      0.84      0.85     71741

